In [ ]:
import kagglehub
path = kagglehub.dataset_download("nelgiriyewithana/world-stock-prices-daily-updating")
print("Path to dataset files:", path)

In [ ]:
import pandas as pd
file_path = "/kaggle/input/world-stock-prices-daily-updating/World-Stock-Prices-Dataset.csv"

df = pd.read_csv(file_path)
print(df.shape)
print(df.columns)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

file_path = "/kaggle/input/world-stock-prices-daily-updating/World-Stock-Prices-Dataset.csv"
df = pd.read_csv(file_path)

df.columns = df.columns.str.strip()
df["Date"] = pd.to_datetime(df["Date"], utc=True)
df = df.sort_values("Date")

df["Ticker"] = df["Ticker"].str.strip().str.upper()

ticker_escolhido = "AMZN" # Selecionar a empresa que quiser basta mudar a sigla dela(desde que tenha no dataset)
df = df[df["Ticker"] == ticker_escolhido].copy()

df["Target"] = (df["Close"].shift(-1) - df["Close"]) / df["Close"]# aq calcula o retorno percentual do dia seguinte
df = df.dropna(subset=["Target"])

# Variáveis escolhidas para "prever" o retorno
X = df[["Open", "High", "Low", "Volume"]]
y = df["Target"]

# Divisão temporal
split = int(len(df) * 0.8)

X_train = X.iloc[:split]
X_test = X.iloc[split:]

y_train = y.iloc[:split]
y_test = y.iloc[split:]


# Treino teste
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(" Retorno percentual")
print("MSE:", mse)
print("R²:", r2)


plt.figure(figsize=(14,6))
plt.plot(y_test.values[:800], label="Retorno Real")
plt.plot(y_pred[:800], label="Retorno Previsto")
plt.title(f"Retorno Percentual - {ticker_escolhido} (800 primeiros pontos)")
plt.xlabel("Tempo")
plt.ylabel("Retorno")
plt.legend()
plt.show()
# o modelo preve a variação percentual entre o fechamento do dia seguinte e o fechamento atual
# no entanto o MSE baixo e o R² mais próximo a 0 é justificado e esperado devido ao comportamento quase aleatório dos retornos financeiros
# e não há grande variação por ser diário e não mensal ou anual